In [1]:
# dependecies
import pandas as pd

In [2]:
# filepath
path_1 = "Data/BicycleSales_2015-20211021_Cleaned.xlsx"
path_2 = "Data/BikeSpecs.xlsx"

# read data
sales_df = pd.read_excel(path_1)
specs_df = pd.read_excel(path_2)

In [3]:
# rename specs Object column to Material
specs1_df = specs_df.rename(columns={"Object":"Material"})

# merge specs into sales data frame
merged_df = pd.merge(sales_df, specs1_df, on="Material", how="left")

In [4]:
# reduce down to necessary columns
reduced_df = merged_df[["Material", "Sold-to pt", "Month", "Invoiced Quantity", 
                        "Product hierarchy.1", "Application_x", "Tire Size", "Ply Construction",
                        "Bead", "Color", "TPI - Threads per inch", "Compound", "Technology"
                       ]]

# rename columns
reduced_df = reduced_df.rename(columns={"Product hierarchy.1":"Product hierarchy",
                                        "Application_x":"Application"})

In [10]:
# split tire size by diameter (before x) and width (after x)
reduced_df[["Diameter", "Width"]] = reduced_df["Tire Size"].str.split("X", n=1, expand=True)

reduced_df.head()

,Material,Sold-to pt,Month,Invoiced Quantity,Product hierarchy,Application,Tire Size,Ply Construction,Bead,Color,TPI - Threads per inch,Compound,Technology,Diamter,Width,Diameter
0,TB29688100,1025073,01/2015,1,M143 CREEPY CRAWLER,MODIFIED TRIALS,20X2.00,Single Ply,Wire,Black,60,Super Tacky,ST,20,2.00,20
1,TB73301000,1025073,01/2015,6,M147 DTH,URBAN,26X2.30,Single Ply,Foldable,Black,60,Single,,26,2.30,26
2,TB96802000,1025073,01/2015,2,M148 GRIFTER,URBAN,29X2.50,Single Ply,Wire,Black,60,Single,,29,2.50,29
3,TB96651200,1025073,01/2015,2,M149 TORCH,URBAN,29X2.10,Single Ply,Foldable,Black,120,Single,SilkWorm,29,2.10,29
4,TB35790100,1025073,01/2015,0,M151 SNYPER,BMX,20X2.25,Single Ply,Foldable,Black,60,Dual,EXO,20,2.25,20


In [19]:
reduced_df[["Application","Tire Size","Diameter","Width"]].to_excel('Output/output.xlsx')

In [ ]:
# split Technology into individual components (delimited by /) and return new columns for
# Protection (EXO, EXO+, MaxxShield, SilkWorm, SilkShield, DD, DH) and Tubeless Ready (TR)

In [ ]:
# group by material create a time series for each material for invoiced quantity by month